<a href="https://colab.research.google.com/github/marina-miyamoto/hw7/blob/master/%E5%AE%BF%E9%A1%8C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define Game class
Mostly copied from https://github.com/step2019/hw6/blob/master/python/main.py

In [0]:
import copy
import json

NEW_GAME_JSON = """
{"board":{"Pieces":[[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
[0,0,0,2,1,0,0,0],[0,0,0,1,2,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]],"Next":1}}
"""

# Reads json description of the board and provides simple interface.
class Game(object):
    # Takes json or a board directly.
    def __init__(self, body=None, board=None):
        if board:
            self._board = board
        else:
            game = json.loads(body or NEW_GAME_JSON)
            self._board = game["board"]
            

    # Returns piece on the board.
    # 0 for no pieces, 1 for player 1, 2 for player 2.
    # None for coordinate out of scope.
    def Pos(self, x, y):
        return Pos(self._board["Pieces"], x, y)

    # Returns who plays next.
    def Next(self):
        return self._board["Next"]

    # Returns the array of valid moves for next player.
    # Each move is a dict
    #   "Where": [x,y]
    #   "As": player number
    def ValidMoves(self):
        moves = []
        for y in range(1, 9):
            for x in range(1, 9):
                move = {"Where": [x, y], "As": self.Next()}
                if self.NextBoardPosition(move):
                    moves.append(move)
        return moves

    # Helper function of NextBoardPosition.  It looks towards
    # (delta_x, delta_y) direction for one of our own pieces and
    # flips pieces in between if the move is valid. Returns True
    # if pieces are captured in this direction, False otherwise.
    def __UpdateBoardDirection(self, new_board, x, y, delta_x, delta_y):
        player = self.Next()
        opponent = 3 - player
        look_x = x + delta_x
        look_y = y + delta_y
        flip_list = []
        while Pos(new_board, look_x, look_y) == opponent:
            flip_list.append([look_x, look_y])
            look_x += delta_x
            look_y += delta_y
        if Pos(new_board, look_x, look_y) == player and len(flip_list) > 0:
            # there's a continuous line of our opponents
            # pieces between our own pieces at
            # [look_x,look_y] and the newly placed one at
            # [x,y], making it a legal move.
            SetPos(new_board, x, y, player)
            for flip_move in flip_list:
                flip_x = flip_move[0]
                flip_y = flip_move[1]
                SetPos(new_board, flip_x, flip_y, player)
            return True
        return False

    # Takes a move dict and return the new Game state after that move.
    # Returns None if the move itself is invalid.
    def NextBoardPosition(self, move):
        x = move["Where"][0]
        y = move["Where"][1]
        if self.Pos(x, y) != 0:
            # x,y is already occupied.
            return None
        new_board = copy.deepcopy(self._board)
        pieces = new_board["Pieces"]

        if not (self.__UpdateBoardDirection(pieces, x, y, 1, 0)
                | self.__UpdateBoardDirection(pieces, x, y, 0, 1)
                | self.__UpdateBoardDirection(pieces, x, y, -1, 0)
                | self.__UpdateBoardDirection(pieces, x, y, 0, -1)
                | self.__UpdateBoardDirection(pieces, x, y, 1, 1)
                | self.__UpdateBoardDirection(pieces, x, y, -1, 1)
                | self.__UpdateBoardDirection(pieces, x, y, 1, -1)
                | self.__UpdateBoardDirection(pieces, x, y, -1, -1)):
            # Nothing was captured. Move is invalid.
            return None
        # Something was captured. Move is valid.
        new_board["Next"] = 3 - self.Next()
        return Game(board=new_board)


    def Pass(self):
        if len(self.ValidMoves()) > 0:
            return None
        new_board = copy.deepcopy(self._board)
        new_board["Next"] = 3 - self.Next()
        return Game(board=new_board)


# Returns piece on the board.
# 0 for no pieces, 1 for player 1, 2 for player 2.
# None for coordinate out of scope.
#
# Pos and SetPos takes care of converting coordinate from 1-indexed to
# 0-indexed that is actually used in the underlying arrays.
def Pos(board, x, y):
    if 1 <= x and x <= 8 and 1 <= y and y <= 8:
        return board[y-1][x-1]
    return None


# Set piece on the board at (x,y) coordinate
def SetPos(board, x, y, piece):
    if x < 1 or 8 < x or y < 1 or 8 < y or piece not in [0,1,2]:
        return False
    board[y-1][x-1] = piece


def PrettyMove(move):
    m = move["Where"]
    return '%s%d' % (chr(ord('A') + m[0] - 1), m[1])


# STEP Reversi - Machine Learning

## Train model

In [2]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

model = tf.keras.Sequential([
    tf.keras.layers.Reshape(target_shape=(8, 8, 3, 1), input_shape=(8, 8, 3)),
    tf.keras.layers.Conv3D(10, (4,4, 4), padding='same', activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling3D(pool_size=(2,2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 8, 8, 3, 1)        0         
_________________________________________________________________
conv3d (Conv3D)              (None, 8, 8, 3, 10)       650       
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 3, 10)       0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 4, 4, 1, 10)       0         
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               20608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [4]:
import urllib
import sys
dataset_url = 'https://storage.googleapis.com/step-reversi-dataset/reps3/games-0-of-8765.json'

for line in urllib.request.urlopen(dataset_url):
    if line.strip() == "":
        continue
    data = json.loads(line)
    sys.stdout.write('.')
    history = data['BoardHistory']
    boards = [board['Pieces'] for board in history]
    black_wins = [1 if data['Winner'] == 1 else 0] * len(history)
    one_hot_boards = tf.keras.utils.to_categorical(boards)
    # TODO: データセットのダウンロードと訓練は分けたほうが良いです。
    model.fit(one_hot_boards, black_wins, epochs=10, verbose = 0) # 長時間やるときはverbose=0 を！！

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Try your model against App Engine AI

In [5]:
import sys

def reverse_black_white(boards):
    return [[[(3 - c) % 3 for c in r] for r in b] for b in boards]

def move_by_url(g, url):
    """Apply move by fetching next move from the given url."""
    data = {'board': g._board}
    headers = {'Content-Type': 'application/json'}
    req = urllib.request.Request(url, json.dumps(data).encode(), headers)
    with urllib.request.urlopen(req) as res:
        body = res.read()
    if body == b'PASS':
        return g.Pass()
    elif body[0] == b'['[0]:
        (x, y) = json.loads(body)
    else:
        x = body[0] - b'A'[0] + 1
        y = body[1] - b'1'[0] + 1
    for move in g.ValidMoves():
        if move['Where'][0] == x and move['Where'][1] == y:
            return g.NextBoardPosition(move)

def move_by_model(g, model):
    """Apply move by evaluating all valid moves by the given model."""
    valid_moves = g.ValidMoves()
    if len(valid_moves) == 0:
        return g.Pass()
    boards = []
    for valid_move in valid_moves:
        next = g.NextBoardPosition(valid_move)
        boards.append(next._board['Pieces'])
    if g._board['Next'] != 1:
        boards = reverse_black_white(boards)
    scores = model.predict(tf.keras.utils.to_categorical(boards))
    index = np.argmax(scores.reshape([-1]))
    return g.NextBoardPosition(valid_moves[index])

# Play game!
g = Game()
for i in range(120):
    sys.stdout.write('.')
    # Black turn
    no_move1 = len(g.ValidMoves()) == 0
    g = move_by_model(g, model)
    # White turn
    no_move2 = len(g.ValidMoves()) == 0
    g = move_by_url(g, 'http://l0.krisp-othello.appspot.com')
    # If both passed, game is finished.
    if no_move1 and no_move2: break
print()

# Print results
counts = [0, 0, 0]
for r in g._board['Pieces']:
    for i in r: counts[i] += 1
print('empty', 'black', 'white')
print(counts)
g._board['Pieces']

...............................
empty black white
[0, 30, 34]


[[2, 1, 1, 1, 1, 1, 1, 2],
 [2, 1, 1, 2, 1, 1, 1, 2],
 [2, 1, 1, 1, 2, 2, 1, 2],
 [2, 1, 1, 1, 1, 2, 2, 2],
 [2, 1, 2, 1, 1, 1, 2, 2],
 [2, 1, 2, 2, 1, 1, 1, 2],
 [2, 2, 1, 2, 2, 1, 1, 2],
 [2, 2, 2, 2, 2, 2, 2, 2]]

## Download your model

In [0]:
from google.colab import files
model.save('model.h5')
files.download('model.h5')

## Load saved model

In [0]:
from google.colab import files
uploaded_files = files.upload()
for filename in uploaded_files.keys():
    model = tf.keras.models.load_model(filename)
    model.summary()
    break # Ignore except for the first file if multiple files are uploaded.